### General info of the datasets:

Questions contains the title, body, creation date, score, and owner ID for each Python question.

Answers contains the body, creation date, score, and owner ID for each of the answers to these questions. The ParentId column links back to the Questions table.

Tags contains the tags on each question besides the Python tag.

# Reading and Cleaning data

In [1]:
import pandas as pd

In [2]:
df_tags = pd.read_csv('datasets/Tags.csv')
print(df_tags.info())
print(df_tags.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885078 entries, 0 to 1885077
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 28.8+ MB
None
    Id        Tag
0  469     python
1  469        osx
2  469      fonts
3  469  photoshop
4  502     python


In [3]:
df_tags_filtered = df_tags[df_tags['Tag']=='python']
df_tags_filtered.head()

,Id,Tag
0,469,python
4,502,python
8,535,python
11,594,python
16,683,python


In [4]:
df_questions = pd.read_csv('datasets/Questions.csv', encoding='ISO-8859-1')
print(df_questions.info())
print(df_questions.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607282 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            607282 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  607282 non-null  object 
 3   Score         607282 non-null  int64  
 4   Title         607282 non-null  object 
 5   Body          607282 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 27.8+ MB
None
    Id  OwnerUserId          CreationDate  Score  \
0  469        147.0  2008-08-02T15:11:16Z     21   
1  502        147.0  2008-08-02T17:01:58Z     27   
2  535        154.0  2008-08-02T18:43:54Z     40   
3  594        116.0  2008-08-03T01:15:08Z     25   
4  683        199.0  2008-08-03T13:19:16Z     28   

                                               Title  \
0  How can I find the full path to a font from it...   
1            Get a preview JPEG of a PDF on W

## Clean out html elements from the text data

In [5]:
from bs4 import BeautifulSoup

In [6]:
# Define a function to extract text from HTML tags using BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

# Apply the function to the 'Text' column of the dataframe
df_questions['Body'] = df_questions['Body'].apply(remove_html_tags)

# Print cleaned dataframe
print(df_questions.head())

KeyboardInterrupt: 